# Sistemas de ecuaciones sobredeterminados

Este notebook repasa brevemente las herramientas apropiadas para resolver varios tipos de sistemas de ecuaciones lineales que surgen al estimar modelos a partir de observaciones.

Resolvamos el sistema de ecuaciones

$$
\begin{align*}
x + 2y &= 3\\
3x+4y  &= 5
\end{align*}
$$

o, en forma matricial $A X = B$:

$$\begin{bmatrix}1 & 2 \\ 3 & 4\end{bmatrix} X = \begin{bmatrix}3 \\ 5 \end{bmatrix}
$$

In [1]:
import numpy as np

In [2]:
a = np.array([[1,2]
             ,[3,4]])

b = np.array([3,5])

In [3]:
x = np.linalg.solve(a,b)

In [4]:
x

array([-1.,  2.])

In [5]:
a @ x

array([3., 5.])

También se puede resolver con la inversa de la matriz de coeficientes. $x=A^{-1}B$

In [6]:
np.linalg.inv(a)

array([[-2. ,  1. ],
       [ 1.5, -0.5]])

In [7]:
np.linalg.inv(a) @ b

array([-1.,  2.])

Pero normalmente se recomienda `solve` porque es más eficiente y estable numéricamente.

### Múltiples lados derechos

Cuando hay múltiples lados derechos podemos ponerlos en una matriz y entonces tenemos una ecuación $AX=B$ donde $A$, $B$, y $X$ son matrices. Por ejemplo:

$$\begin{bmatrix}1 & 2 \\ 3 & 4\end{bmatrix} X = \begin{bmatrix}3 & 1 & 2\\ 5 & 7 & 2\end{bmatrix}
$$

Son tres sistemas distintos que comparten la misma matriz de coeficientes $A$.

In [8]:
a = np.array([[1,2]
             ,[3,4]])

b = np.array([[3,1,2]
             ,[5,7,2]])

In [9]:
x = np.linalg.solve(a,b)
x

array([[-1.,  5., -2.],
       [ 2., -2.,  2.]])

In [10]:
a @ x

array([[3., 1., 2.],
       [5., 7., 2.]])

Se puede precomputar una estructura (la [descomposición LU](https://en.wikipedia.org/wiki/LU_decomposition)) que permite resolver rápidamente cualquier lado derecho.

### Número de soluciones

El sistema $AX=B$ tendrá solución única si $A$ tiene rango completo. Si hay más ecuaciones (independientes) que incógnitas el sistema será incompatible, y si hay menos, tendrá múltiples soluciones.

### Sistema homogéneo

Si $B = 0$ tenemos un sistema *homogéneo*: buscamos un vector $X \neq \vec 0$ que consiga $AX=\vec 0$. La solución será un elemento del *espacio nulo* de $A$. Para que exista una solución no trivial la matriz $A$ no pude ser de rango completo.

Por ejemplo, resolvamos

$$\begin{bmatrix}1 & 2 & 3 \\ 3 & -4 & 7\end{bmatrix} X = \begin{bmatrix}0 \\ 0 \end{bmatrix}
$$

In [11]:
a = np.array([[1,2,3],
              [3,-4,7]])

# calcula el espacio nulo de una matriz
# suponiendo que tiene dimensión 1
def null1(M):
    u,s,vt = np.linalg.svd(M)
    return vt[-1]

x = null1(a)
x

array([-0.93094934, -0.07161149,  0.35805744])

In [12]:
a @ x

array([-8.8817842e-16,  4.4408921e-16])

(Son ceros núméricos.)

### Sistemas sobredeterminados

En muchas aplicaciones surgen sistemas de ecuaciones que tienen más ecuaciones que incógnitas. Esto suele ocurrir cuando se intenta estimar los parámetros de un modelo a partir de un gran número de datos experimentales. Estrictamente hablando esos sistemas son incompatibles pero en muchos casos se puede encontrar una solución aceptable que minimice el error cuadrático.

Por ejemplo, el sistema siguiente no tiene solución:

$$\begin{bmatrix}1 & 2 \\ 3 & 4 \\ 6 & 7\end{bmatrix} X = \begin{bmatrix}3 \\ 5 \\ 10 \end{bmatrix}
$$

In [13]:
a = np.array([[1,2],
              [3,4],
              [6,7]])

b = np.array([3,5,10])

En este caso `np.linalg.solve(a,b)` daría error: "expected square matrix". Pero podemos encontrar una solución aproximada con `np.linalg.lstsq` (*least squares*):

In [14]:
x = np.linalg.lstsq(a,b, rcond=None)[0]
x

array([0.15789474, 1.26315789])

In [15]:
a @ x

array([2.68421053, 5.52631579, 9.78947368])

Esta función devuelve también el error conseguido (suma de residuos cuadráticos) e información sobre lo "bien condicionado" que está el sistema.

Se obtiene la misma solución utilizando la *pseudoinversa*: $A^+ \equiv (A^T A)^{-1}A^T$

In [16]:
pinv = np.linalg.inv(a.T @ a) @ a.T
pinv

array([[-1.13157895, -0.44736842,  0.57894737],
       [ 0.94736842,  0.42105263, -0.36842105]])

In [17]:
pinv @ b

array([0.15789474, 1.26315789])

Pero si solo vamos a usarla para multiplicar por $B$ es mejor usar directamente `np.linalg.lstsq`.

### Sistemas homogéneos sobredeterminados

Finalmente, también nos encontraremos con sistemas homogéneos sobredeterminados. Por ejemplo:
    
$$\begin{bmatrix}10 & 20 & 50 \\ 10 & 21  & 49 \\ 9 & 20 & 51 \\ 11 & 19 & 50\end{bmatrix} X = \begin{bmatrix}0 \\ 0 \\ 0\\0\end{bmatrix}
$$

In [18]:
a = np.array([[10, 20, 50],
              [10, 21, 49],
              [ 9, 20, 51],
              [11, 19, 50]])

x = null1(a)
x

array([ 0.78774705,  0.50132179, -0.35795398])

La función `null1` definida más arriba obtiene el vector de norma 1 que minimiza $\left\rVert AX \right\rVert$. Se basa en la [descomposición SVD](https://en.wikipedia.org/wiki/Singular_value_decomposition), la herramienta más importantes del Álgebra Lineal Numérica.

In [19]:
a @ x

array([ 0.00620698,  0.86548276, -1.13949405,  0.29263224])

Este vector no "anula" perfectamente a la matriz pero consigue un residuo pequeño.

### Ejercicios:

- Encuentra la recta que mejor se ajusta a una colección de puntos 2D.

- Encuentra la ecuación de una elipse que pase cerca de un conjunto de puntos 2D.

### Sistemas sobredeterminados y generalización

El problema fundamental en  *Machine Learning* es encontrar un modelo de los ejemplos de entrenamiento que resuelva correctamente la mayoría de casos futuros no vistos previamente.

Si estos ejemplos son representativos del problema y sobredeterminan los parámetros libres del modelo, y además el proceso de aprendizaje (optimización) consigue una buena solución, cabe esperar que esta solución "generalice" y resuelva bien casos futuros. Pero si los parámetros no están sobredeterminados, es seguro que encontraremos una solución perfecta para los datos de entrenamiento, sean cuales sean las etiquetas de clase que se les asignen: es un aprendizaje puramente memorístico, sin ninguna garantía sobre los resultados en futuras instancias del problema. Es necesario controlar la capacidad de las máquinas de aprendizaje (véase el apartado "Regularización" del capítulo sobre [Machine Learning](machine-learning.ipynb#Regularización)).